# README.md

In [53]:
## Table of Contents

In [1]:
import numpy as np
from pandas import DataFrame

import pandas as pd
import sys, getopt
import csv
from datetime import datetime
from datetime import timedelta, date
import itertools

from matplotlib import pyplot
from pylab import *
import matplotlib.pyplot as plt

%matplotlib inline  
import seaborn as sns

# Step1, read from spreadsheet, clean up, and get statistics

In [50]:
df1=pd.read_excel("./data/270pt_adt.xlsx", header=0)
df1.replace(r'', np.NaN)
df1.fillna('', inplace=True)
#df1['initial_ADT_end_date'] = pd.to_datetime(df1['initial_ADT_end_date'])

df_ttest=pd.read_excel("./data/pca_tresults_20210119_BP_fixed.xlsx", header=0)
#df3=pd.read_csv("just_start_empty.csv", header=0)
df_ttest.replace(r'', np.NaN)
df2=df_ttest
df_ttest[:3]

,person_id,cancer_type_id,year_of_diagnosis,month_of_diagnosis,day_of_diagnosis,dx_date,result_date,loinc_code,source_test_name,test_results
0,2,2,2003,1.0,7.0,2003-01-07,2006-09-05,49041-7,TESTOSTERONE,351.0
1,2,2,2003,1.0,7.0,2003-01-07,2007-01-09,49041-7,TESTOSTERONE,304.0
2,2,2,2003,1.0,7.0,2003-01-07,2007-05-17,49041-7,TESTOSTERONE,259.0


In [51]:
print(shape(df1))
print(shape(df_test))

(270, 4)
(18643, 10)


In [52]:
df1[:3]

,person_id,initial_ADT_start_date,initial_ADT_end_Date,ADT_duration (days)
0,2,2009-02-26,2009-09-15,201
1,13,2010-06-17,2015-03-17,1734
2,25,2013-11-01,2014-02-18,109


In [53]:
df1_pt_grouped=df1.groupby(["person_id"]).size().reset_index(name='counts')
list_pt=df1_pt_grouped["person_id"].to_list()
print("")
print("Total number of pts")
print(len(list_pt))


Total number of pts
270


In [54]:
list_pt[:10]

[2, 13, 25, 38, 81, 110, 119, 780, 870, 905]

In [55]:
###make sure all pts are included in the T test result table

df_tvalue_grouped=df_test.groupby(["person_id"]).size().reset_index(name='counts')
list_pt_test=df_tvalue_grouped["person_id"].to_list()
print("list of pt ids with t test results")
print(len(list_pt_test))
print("")
print(len(list_pt))

overlap_list=[x for x in list_pt if x in  list_pt_test]
print(len(overlap_list))

list of pt ids with t test results
4490

270
270


In [56]:
df1[:3]
df1.columns=['person_id','initial_ADT_start_date','initial_ADT_end_date','ADT_duration (days)']
df1[:3]

,person_id,initial_ADT_start_date,initial_ADT_end_date,ADT_duration (days)
0,2,2009-02-26,2009-09-15,201
1,13,2010-06-17,2015-03-17,1734
2,25,2013-11-01,2014-02-18,109


In [57]:
df2_pt=df2[(df2["person_id"] == 25)]    
df2_pt

,person_id,cancer_type_id,year_of_diagnosis,month_of_diagnosis,day_of_diagnosis,dx_date,result_date,loinc_code,source_test_name,test_results
128,25,2,2013,5.0,10.0,2013-05-10,2013-10-15,49041-7,TESTOSTERONE,31.54
129,25,2,2013,5.0,10.0,2013-05-10,2015-02-18,49041-7,TESTOSTERONE,149.89
130,25,2,2013,5.0,10.0,2013-05-10,2015-07-02,49041-7,TESTOSTERONE,143.30
131,25,2,2013,5.0,10.0,2013-05-10,2015-12-14,49041-7,TESTOSTERONE,171.64
132,25,2,2013,5.0,10.0,2013-05-10,2016-06-03,49041-7,TESTOSTERONE,258.91
133,25,2,2013,5.0,10.0,2013-05-10,2017-01-10,49041-7,TESTOSTERONE,153.81
134,25,2,2013,5.0,10.0,2013-05-10,2017-07-11,49041-7,TESTOSTERONE,172.78
135,25,2,2013,5.0,10.0,2013-05-10,2018-01-30,49041-7,TESTOSTERONE,236.60
136,25,2,2013,5.0,10.0,2013-05-10,2018-07-26,49041-7,TESTOSTERONE,242.20
137,25,2,2013,5.0,10.0,2013-05-10,2019-01-22,49041-7,TESTOSTERONE,148.87


# How many pts have ADT window assginable? 

In [66]:
result_status_list=[]

for y in list_pt:     
#for y in [2, 13,25]:
        
    df1_pt=df1[(df1["person_id"] == y)]
   
    if(len(df1_pt.initial_ADT_end_date)==0):        
        b='NaN'
        #gap='NaN'
        tdate='NaN'
        tvalue='NaN'
        status="NaN"
        N_t_dates='NaN'
        N_t_dates_afterADT='NaN'

    if(len(df1_pt.initial_ADT_end_date)>0):
        b=df1_pt['initial_ADT_end_date'].to_list()[0]
        #print(b)
        df2_pt=df2[(df2["person_id"] == y)]    
        each_pt_tdates = df2_pt['result_date'].to_list()
        #print(each_pt_tdates)
        ##get rid of nan
        cleaned_a=[]
        cleaned_a= [x for x in each_pt_tdates if str(x) != 'NaN']
        cleaned_aa= [x for x in cleaned_a if str(x) != 'nan']
        cleaned_aaa= [x for x in cleaned_aa if str(x) != 'NaT']
        N_t_dates=len(cleaned_aaa)

        #only keep the ones after ADT end date including on the day
        cleaned_aaaa= [x for x in cleaned_aaa if x >=b]
        
        start_sorted_bin=sorted(cleaned_aaaa)
        N_t_dates_afterADT=len(cleaned_aaaa)
   
        if(len(start_sorted_bin)==0):
            tdate="NaN" 
            tvalue="NaN"
            status="NaN"
            
#get the 300

        if(len(start_sorted_bin)>0):
            df3_pt=df2_pt[df2_pt.test_results >=300][df2_pt.result_date >=b]
            df4_pt=df2_pt[df2_pt.test_results <300][df2_pt.result_date >=b]

            if not df3_pt.empty:
                status ="Y"
                tdate=df3_pt['result_date'].to_list()[0]
                tvalue=df3_pt['test_results'].to_list()[0]

                #if not df4_pt.empty:
            if df3_pt.empty:
                status ="N"
                tdate=df4_pt['result_date'].to_list()[-1]
                tvalue=df4_pt['test_results'].to_list()[-1]
    
    #print(y, b, tdate, tvalue,status, N_t_dates,N_t_dates_afterADT)
    result_status_per_pt=[y, b, tdate, tvalue, status,N_t_dates,N_t_dates_afterADT]
    result_status_list.append(result_status_per_pt)

/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [67]:
df_result_status=pd.DataFrame(result_status_list)
df_result_status.columns=(['person_id', 'initial_ADT_end_date', 'Tdate_rec_norec', 'Tvalue_rec_norec', 'Status',"N_T_dates","N_T_dates_afterADT"])
df_result_status

,person_id,initial_ADT_end_date,Tdate_rec_norec,Tvalue_rec_norec,Status,N_T_dates,N_T_dates_afterADT
0,2,2009-09-15,2017-04-11,4.80,N,82,61
1,13,2015-03-17,2017-12-14,4.33,N,23,15
2,25,2014-02-18,2020-01-21,238.49,N,12,11
3,38,2016-07-18,2018-04-16,317.16,Y,20,13
4,81,2016-06-09,2017-04-03,309.59,Y,12,10
...,...,...,...,...,...,...,...
265,186766,2020-02-03,2020-09-02,730.60,Y,10,4
266,187363,2019-06-19,2020-02-24,366.97,Y,9,4
267,187900,2020-02-03,2020-03-02,13.86,N,6,2
268,189405,2019-07-29,2020-02-10,73.90,N,10,6


In [68]:
df_rec_pt=df_result_status[df_result_status.Status=="Y"]
df_nonrec_pt=df_result_status[df_result_status.Status=="N"]

In [69]:
df_result_status.to_csv("./data/270pt_recovery_status_Mar12.csv", encoding='utf-8', index=False)